In [1]:
import torch

model = torch.load('./model.pth')

C:\Users\admin\AppData\Local\Temp\ipykernel_11204\2085947940.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model.pth')


In [3]:
from torch import nn

class DeepCollaborativeFiltering_target(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(DeepCollaborativeFiltering_target, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)  # Sortie unique pour binaire

    def forward(self, user_ids, item_ids):
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        x = torch.cat([user_emb, item_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))  # Activation sigmoïde pour classification binaire
        return x


In [7]:
# Nombre total d'utilisateurs et d'items dans le domaine cible
num_users_target = 583964  # Utilisateurs communs (source + cible)
num_items_target = 546895

In [8]:
# Chargement du modèle sauvegardé
model_target = DeepCollaborativeFiltering_target(num_users_target, num_items_target)
model_target.load_state_dict(torch.load("./model.pth"))
model_target.eval()
print("Modèle chargé avec succès")

C:\Users\admin\AppData\Local\Temp\ipykernel_11204\3306229806.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_target.load_state_dict(torch.load("./model.pth"))


Modèle chargé avec succès


In [20]:
# Liste des items spécifiques à prédire
user_id = 123

specific_items = list(range(1,10_000))

# Calcul des probabilités pour chaque item
probabilities = []
with torch.no_grad():
    for item_id in specific_items:
        user_tensor = torch.tensor([user_id], dtype=torch.long)
        item_tensor = torch.tensor([item_id], dtype=torch.long)

        # Prédire la probabilité avec le modèle
        probability = model_target(user_tensor, item_tensor).item()
        probabilities.append((item_id, probability))

probabilities = sorted(probabilities, key=lambda x: -x[1])

# # Afficher les résultats
# print("Probabilités pour les items spécifiques :")
# for item_id, prob in probabilities:
#     print(f"Item {item_id}: Click Probability = {prob:.4f}")

probabilities[:10]

[(9228, 0.9735859036445618),
 (9830, 0.9695830941200256),
 (9925, 0.9631091952323914),
 (8238, 0.9584673643112183),
 (9340, 0.9531669616699219),
 (147, 0.9518650770187378),
 (3662, 0.9456021785736084),
 (7689, 0.9375821948051453),
 (2586, 0.933740496635437),
 (749, 0.9274648427963257)]

In [22]:
for i,j in probabilities[:10]:
    print(i)

9228
9830
9925
8238
9340
147
3662
7689
2586
749


In [23]:
from torch import nn
import torch

class DeepCollaborativeFiltering_target(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(DeepCollaborativeFiltering_target, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)  # Sortie unique pour binaire

    def forward(self, user_ids, item_ids):
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        x = torch.cat([user_emb, item_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))  # Activation sigmoïde pour classification binaire
        return x

#---------------------------------------------------
num_users_target = 583964
num_items_target = 546895 

model = DeepCollaborativeFiltering_target(num_users_target, num_items_target)
model.load_state_dict(torch.load("./model.pth"))
model.eval()
#---------------------------------------------------

def recommande(user_id: int, items: list, k: int):
    probabilities = []
    
    with torch.no_grad():
        for item_id in items:
            user_tensor = torch.tensor([user_id], dtype=torch.long)
            item_tensor = torch.tensor([item_id], dtype=torch.long)

            probability = model(user_tensor, item_tensor).item()
            probabilities.append((item_id, probability))

    probabilities = sorted(probabilities, key=lambda x: -x[1])
    
    return probabilities[:k]

C:\Users\admin\AppData\Local\Temp\ipykernel_11204\4155684304.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./model.pth"))


In [26]:
def generate_product(id, score: float=None):
    return {
        'id': id,
        # 'title': f'Title of {id}',
        # 'images': [],
        # 'price': random.randint(100, 10_000) * 0.01,
        # 'rating': random.randint(1, 5),
        # 'description': f'Description of item {id}',
        # 'category': f'Category {round(id % 100)}',
        # 'discountPercentage': random.randint(1, 50),
        # 'score': None if score is None else str(score)
    }

def generate_products(ids):
    return [generate_product(i) for i in ids]

def generate_products_score(ids_scores):
    return [generate_product(i[0], i[1]) for i in ids_scores]


In [29]:
user_id = 1

count = 20

items = list(range(1, 1_000))

# top_items = recommande(user_id, items, count)
# products = generate_products_score(top_items)
# products

recommande(user_id, items, count)